In [1]:
!pip3 install earthengine-api 
!pip3 install earthengine-api 


In [ ]:
import ee # Google Earth Engine
import datetime
#import ipyleaflet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import shapely
import shapely.geometry
# from shapely.geometry import MultiPolygon, Polygon
import warnings
from functools import reduce


In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=s8T_p8LliVAuZDQg6DIWX7tPxp0_XRm_282LHuq1uQY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/0gHJxsNQ1ffLGUheFCSNxN9fk6CdvNgEgAi_Z81y9-xbxU5PEd5lN7k

Successfully saved authorization token.


Identify and load G drive data

In [ ]:
gdrive_project_pathname = '/My Drive/w210'  #@param {type: "string"}

In [5]:
import os.path
from google.colab import drive

# Mount the current user's Google Drive
GOOGLE_DRIVE_MOUNT_POINT = "/content/drive"
print("Mounting Google Drive beneath %s" % GOOGLE_DRIVE_MOUNT_POINT)
drive.mount(GOOGLE_DRIVE_MOUNT_POINT, force_remount = True)

# Build the pathname to the project's folder residing beneath the current user's Google Drive
if not gdrive_project_pathname.startswith("/"):
  gdrive_project_pathname = "/" + gdrive_project_pathname
abs_project_pathname = GOOGLE_DRIVE_MOUNT_POINT + gdrive_project_pathname
print("Project folder: %s" % gdrive_project_pathname)


Mounting Google Drive beneath /content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Project folder: /My Drive/w210


In [6]:
# os.listdir(abs_project_pathname)
df_csv = pd.read_csv(abs_project_pathname + '/labeled_data_points.csv')
df_csv.head()

,system:index,X,Y,b1,constant,.geo
0,00000000000000000002_0,-121.919350,53.216197,0.193,1,NaN
1,00000000000000000003_0,-120.625776,40.136727,0.654,1,NaN
2,00000000000000000004_0,-91.735957,18.289699,0.219,1,NaN
3,00000000000000000006_0,-80.596847,32.519013,0.254,1,NaN
4,00000000000000000007_0,-67.445512,53.000602,0.000,1,NaN


Specify an area of interest

In [ ]:
# area of interest has lat, lon of (-10.883689, -44.005800)
zone_ex1_lat, zone_ex1_lon = 56.88132379,	33.74072207 #-120.6257764,	40.13672689 #-121.9193504,	53.21619743
zone_ex1_edge_len = 0.2

In [ ]:
def satallite_imagery(source, center_lat, center_lon, edge_len,
                      start_date, end_date, 
                      plot_option):
    # Sentinel-2 Level 1-C: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2
    # Sentinel-2 Level 2-A: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
    # Landsat Tier structure: https://developers.google.com/earth-engine/landsat#landsat-collection-structure
    if source == "Sentinel2_1C":
        source_loc = 'COPERNICUS/S2'
    elif source == "Sentinel2_2A":
        source_loc = 'COPERNICUS/S2_SR'
    elif source == "Landsat7":
        source_loc = 'LANDSAT/LE07/C01/T1'
    elif source == "Landsat8":
        source_loc = 'LANDSAT/LC08/C01/T1_SR'
    else:
        raise Exception('Invalid source of satellite imagery')
        
    area_of_interest = ee.Geometry.Polygon([[center_lon-edge_len/2, center_lat-edge_len/2], 
                           [center_lon-edge_len/2, center_lat+edge_len/2], 
                           [center_lon+edge_len/2, center_lat+edge_len/2], 
                           [center_lon+edge_len/2, center_lat-edge_len/2]])
    
    img_collect = (ee.ImageCollection(source_loc)
                 .filterDate(start_date, end_date)
                 .filterBounds(area_of_interest))
    
    # if 'Sentinel' in source:
    #     img_collect = img_collect.filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",100)
    # elif 'Landsat' in source:
    #     img_collect = img_collect.filter(ee.Filter.lt('CLOUD_COVER', 0))

    # assert (img_collect.size().getInfo()>0), "No valid image"
    if img_collect.size().getInfo()>0:
        print("Number of images in the collection: ", img_collect.size().getInfo(), " at: ", center_lat, center_lon)
        
        if 'Sentinel' in source:
            # Reference: https://www.satimagingcorp.com/satellite-sensors/other-satellite-sensors/sentinel-2a/
            band_blue = 'B2' #10m
            band_green = 'B3' #10m
            band_red = "B4"  #10m
            band_nir = 'B8'  #10m
        # Reference: https://landsat.gsfc.nasa.gov/landsat-data-continuity-mission/
        elif 'Landsat7' in source:
            # Reference: https://www.usgs.gov/land-resources/nli/landsat/landsat-7?qt-science_support_page_related_con=0#qt-science_support_page_related_con
            band_blue = 'B1' #30m
            band_green = 'B2' #30m
            band_red = "B3"  #30m
            band_nir = 'B4'  #30m
        elif 'Landsat8' in source:
            # Reference: https://www.usgs.gov/faqs/what-are-best-landsat-spectral-bands-use-my-research?qt-news_science_products=0#qt-news_science_products
            band_blue = 'B2' #30m
            band_green = 'B3' #30m
            band_red = "B4"  #30m
            band_nir = 'B5'  #30m    
        
        def calc_NDVI(img):
            ndvi = ee.Image(img.normalizedDifference([band_nir, band_red])).rename(["ndvi"]).copyProperties(img, img.propertyNames())
            composite = img.addBands(ndvi)
            return composite
        
        def add_ee_layer(self, ee_object, vis_params, name):
            try:    
                if isinstance(ee_object, ee.image.Image):    
                    map_id_dict = ee.Image(ee_object).getMapId(vis_params)
                    folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
                elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
                    ee_object_new = ee_object.mosaic()
                    map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                    folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
                elif isinstance(ee_object, ee.geometry.Geometry):    
                    folium.GeoJson(
                    data = ee_object.getInfo(),
                    name = name,
                    overlay = True,
                    control = True
                ).add_to(self)
                elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
                    ee_object_new = ee.Image().paint(ee_object, 0, 2)
                    map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                    folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                ).add_to(self)

            except:
                print("Could not display {}".format(name))

        # Add EE drawing method to folium.
        folium.Map.add_ee_layer = add_ee_layer
        
        img_display = img_collect.map(calc_NDVI).median()
        
        # Create a folium map object.
        myMap = folium.Map(location=[center_lat, center_lon], zoom_start=8)
        
        if plot_option == 'RGB':
            if 'Sentinel' in source:
                visParams = {'min':0, 'max':3000}
            elif 'Landsat' in source:
                visParams = {'min':0, 'max':255}
            myMap.add_ee_layer(img_display.select(band_red, band_green, band_blue), visParams, name=source+' '+plot_option)
        elif plot_option == 'VI':
            visParams = {'min':0, 'max':1, 'palette': ['red', 'yellow', 'green']}
            myMap.add_ee_layer(img_display.select("ndvi"), visParams, name=source+' '+plot_option)

        # Add a layer control panel to the map.
        myMap.add_child(folium.LayerControl())
        
        return myMap
    else:
      print("No valid image for: ", center_lat, center_lon)
      # return None

In [19]:

satallite_imagery(source="Sentinel2_1C", center_lat=zone_ex1_lat, center_lon=zone_ex1_lon, edge_len=zone_ex1_edge_len,
                  start_date='2015-06-01', end_date='2016-06-30',
                  plot_option='VI')

Number of images in the collection:  30


In [8]:
zone_ex1_edge_len = 0.2
for recs in df_csv.index:
  # zone_ex1_lat, zone_ex1_lon = df_csv['X'][recs], df_csv['Y'][recs]
  zone_ex1_lat, zone_ex1_lon = df_csv['Y'][recs], df_csv['X'][recs]
  satallite_imagery(source="Sentinel2_1C", center_lat=zone_ex1_lat, center_lon=zone_ex1_lon, edge_len=zone_ex1_edge_len,
                  start_date='2015-07-01', end_date='2016-06-30',
                  plot_option='VI')

Number of images in the collection:  66  at:  53.21619743124045 -121.91935036070149
Number of images in the collection:  42  at:  40.13672689446021 -120.62577635156937
Number of images in the collection:  19  at:  18.28969918467345 -91.73595681428554
Number of images in the collection:  33  at:  32.519013285126675 -80.59684729120347
Number of images in the collection:  70  at:  53.000601763051755 -67.44551153169368
Number of images in the collection:  93  at:  52.497545203944824 -76.35679915015933
Number of images in the collection:  12  at:  -5.138363425163675 -48.68868839927808
Number of images in the collection:  58  at:  -40.567918230837606 -69.60146821358053
Number of images in the collection:  28  at:  -13.83405537544064 -48.25749706290072
Number of images in the collection:  44  at:  -1.7606979568742671 -60.40271970419664
Number of images in the collection:  34  at:  -20.01446453018295 -62.77427205427218
Number of images in the collection:  28  at:  -15.918146834597936 -51.06024

In [10]:
#  var latlon_image = ee.Image.pixelLonLat()    .clipToCollection(samples).select(['longitude','latitude'],['X','Y']);
zone_ex1_edge_len = 0.2
for recs in df_csv.index:
  # zone_ex1_lat, zone_ex1_lon = df_csv['X'][recs], df_csv['Y'][recs]
  zone_ex1_lat, zone_ex1_lon = df_csv['Y'][recs], df_csv['X'][recs]
  m = satallite_imagery(source="Sentinel2_1C", center_lat=zone_ex1_lat, center_lon=zone_ex1_lon, edge_len=zone_ex1_edge_len,
                        start_date='2015-07-01', end_date='2016-06-30',
                        plot_option='VI')
  display(m)

Number of images in the collection:  66  at:  53.21619743124045 -121.91935036070149


Number of images in the collection:  42  at:  40.13672689446021 -120.62577635156937


Number of images in the collection:  19  at:  18.28969918467345 -91.73595681428554


Number of images in the collection:  33  at:  32.519013285126675 -80.59684729120347


Number of images in the collection:  70  at:  53.000601763051755 -67.44551153169368


...

Number of images in the collection:  29  at:  31.08170883053544 69.24214209993272


Number of images in the collection:  34  at:  -29.78813482140335 121.20069813340584


Number of images in the collection:  14  at:  -26.91352591222088 148.79694366155755


Number of images in the collection:  46  at:  -24.110782225767966 135.50187745658866


Number of images in the collection:  21  at:  -24.03891700303841 140.31684737946924
